# Assigment 8

In [ ]:
from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }a
</style>
"""))

In [ ]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import matplotlib.pyplot as plt 
import chardet

In [ ]:
import geopandas as gpd
from geopandas import GeoSeries
from shapely.geometry import Point, LineString
import folium 
from folium import Marker, GeoJson
from folium.plugins import MarkerCluster, HeatMap

In [ ]:
#!pip install mapclassify
import mapclassify

### 8.1. Import the data located at this link. It has information on people infected with dengue at the district level for 2015 to 2021.

In [ ]:
# Read the content of the file in binary mode ('rb')
with open('../../_data/data_dengue_peru.csv', 'rb') as f:
    data = f.read()
    
# Detect the encoding of the file
detected_encoding = chardet.detect(data)['encoding']

# Assign the detected encoding to a variable
charenc = detected_encoding

# Print the detected encoding
print("La codificación detectada es:", charenc)

In [ ]:
# Import csv file from panel dengue information at department, province, and district level
dengue_data = pd.read_csv( r'../../_data/data_dengue_peru.csv', encoding = charenc,  thousands = ",", dtype= {"Ubigeo":"str"})
dengue_data

### 8.2. Generate ubigeo for Departments and Provinces taking the first two and four numbers. Hint: Use this code.

In [ ]:
# Generate a column called 'dep_ubigeo' taking the first two digits from the 'Ubigeo' column
dengue_data['dep_ubigeo'] = dengue_data['Ubigeo'].str[:2]

# Generate another column called 'prov_ubigeo' taking the first four digits from the 'Ubigeo' column
dengue_data['prov_ubigeo'] = dengue_data['Ubigeo'].str[:4]

# # Display the 'dengue_data' dataframe with the new columns
dengue_data

### 8.3. Use geopandas to plot the number of cases in 2021 by the district using a continuous legend. Do not forget to indicate the color of NA values. Use this shapefile.

#### 8.3.1 Filter the 'dengue_data' to get only the cases in the year 2021

In [ ]:
dengue_casos_2021 = dengue_data[dengue_data['Año'] == 2021]
dengue_casos_2021

#### 8.3.2 Select the relevant columns and create the 'Total_casos' column

In [ ]:
total_dengue_casos2021 = dengue_casos_2021.groupby( ['Departamento', 'Provincia','Distrito', 'Ubigeo', 'dep_ubigeo', 'prov_ubigeo'], as_index = False).agg(Total_casos=('Casos', 'sum'))
total_dengue_casos2021

#### 8.3.3 Upload shape file at district level

In [ ]:
distritos_shp = gpd.read_file(r"../../_data/LIMITE_DISTRITAL_2020_INEI/INEI_LIMITE_DISTRITAL.shp")
distritos_shp

In [ ]:
# Get the column names of the 'distritos_shp' dataframe
distritos_shp.columns

In [ ]:
#Check unique values of the 'UBIGEO' column in the 'distritos_shp'
distritos_shp['UBIGEO'].is_unique

In [ ]:
# Calculate the size
distritos_shp['UBIGEO'].unique().size

In [ ]:
# Select the important columns 'UBIGEO' and 'geometry'
distritos_shp = distritos_shp[['UBIGEO', 'geometry']]

# Rename the column 'UBIGEO' to 'Ubigeo'
distritos_shp = distritos_shp.rename(columns={'UBIGEO':'Ubigeo'})
distritos_shp

In [ ]:
#  WGS-84 (longitud-latitud projection) 
distritos_shp.crs
# System of coordenates: CRS (due to where we are)

###  Merge: 'total_dengue_casos2021' & 'distritos_shp'

In [ ]:
# Merge 'distritos_shp' dataFrame with 'total_dengue_casos2021' dataframe based on 'Ubigeo' column, keeping all rows from 'distritos_shp'
dengue_dist_2021 = pd.merge(distritos_shp, total_dengue_casos2021, how='left', on=['Ubigeo'])

# Drop the columns 'Departamento', 'Provincia', and 'Distrito' from the merged dataframe
dengue_dist_2021 = dengue_dist_2021.drop(columns=['Departamento','Provincia','Distrito'])

# Convert the 'Ubigeo' column to integer type after converting it to string to handle potential leading zeros
dengue_dist_2021['Ubigeo'] = dengue_dist_2021['Ubigeo'].astype(str).astype(int)
dengue_dist_2021

In [ ]:
# Plot the'dengue_dist_2021' dataFrame based on the 'Total_casos' column, using the 'Reds' colormap,
# with a specified figure size of 20x20 inches, dashed line style for district boundaries, and black edge color.
# Enable the legend, specifying a user-defined classification scheme with defined bins for data classification.
# Set the color for missing values and specify legend properties such as position, font size, title, and frame.
dengue_dist_2021.plot( column='Total_casos', cmap='YlGn', figsize=(20, 20), linestyle='--',
                      edgecolor='black', 
                      legend = True,
              scheme = "User_Defined", 
          missing_kwds= dict(color = "#824b0c",), 
          classification_kwds = dict( bins = [ 20, 60, 100, 140, 180, 220] ), 
          legend_kwds=dict(  loc='upper left',
                            bbox_to_anchor=(1.01, 1),
                            fontsize='x-large',
                            title= "Number of dengue cases by district in 2021", 
                            title_fontsize = 'x-large', 
                            frameon= False ) )